Загрузим нужные библиотеки

In [1]:
! python -m spacy download ru_core_news_sm

                                              0.0/15.3 MB ? eta -:--:--
                                              0.0/15.3 MB 1.3 MB/s eta 0:00:12
                                              0.0/15.3 MB 1.3 MB/s eta 0:00:12
                                             0.1/15.3 MB 871.5 kB/s eta 0:00:18
                                             0.1/15.3 MB 722.1 kB/s eta 0:00:21
                                             0.2/15.3 MB 919.0 kB/s eta 0:00:17
                                             0.2/15.3 MB 981.9 kB/s eta 0:00:16
                                              0.3/15.3 MB 1.2 MB/s eta 0:00:13
     -                                        0.4/15.3 MB 1.4 MB/s eta 0:00:11
     -                                        0.5/15.3 MB 1.5 MB/s eta 0:00:10
     -                                        0.6/15.3 MB 1.6 MB/s eta 0:00:10
     -                                        0.7/15.3 MB 1.6 MB/s eta 0:00:09
     --                                       0.8/15.3 

In [2]:
import pandas as pd
import unicodedata
import re
import spacy
import json
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import json
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

# Подготовка данных

Загрузим данные

In [3]:
data = pd.read_csv('csv\\full_df.csv', sep=';')
data.head(3)
data = data.drop(columns='Unnamed: 0', axis=1)

Отфильтруем тексты по количеству символов. Оставим только 100+.

In [4]:
data['text_len'] = data.loc[:, 'text'].apply(lambda x: len(x))
data_filtered_by_text_len = data.query('text_len > 100')
print(f'Количество постов с 100+ символами: {data_filtered_by_text_len.shape[0]}')
data_filtered_by_text_len.head(3)

Количество постов с 100+ символами: 8556


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Оставим только тексты содержащие кириллицу

In [5]:
data_cyrillic = data_filtered_by_text_len[data_filtered_by_text_len['text'].apply(lambda x: re.match(r'[А-Яа-я]+', x) is not None)]

print(f'Текстов на кириллице: {data_cyrillic.shape[0]}')
data_cyrillic.head(3)

Текстов на кириллице: 1960


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Удалим дубликаты текстов

In [6]:
data_dd = data_cyrillic.drop_duplicates('text')
print(f'Осталось {data_dd.shape[0]} строк')
data_dd.head(3)

Осталось 1862 строк


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
6,Talent Acquisition Manager | Recruitment Lead ...,https://www.linkedin.com/in/dariaivanova,С трепетом готов представить свой продукт. Реф...,8.0,0.0,0.0,841


# Подготовка к моделированию

In [7]:
texts = data_dd["text"].tolist()
texts[0]

'Всем привет!Выпустил свою первую статью на хабр!https://lnkd.in/dt9N6D7BСтатья про историю и технологии разработки игры https://guess-word.com и как мы создали игру с элементами машинного обучения и вышли в ноль за 2 месяцаПри\xa0внимательном прочтении вы даже сможете запустить первую версию игры!'

Проведем лемматизацию

In [8]:
if os.path.exists('lemmas.json'):
    with open("lemmas.json") as f:
        data_lemmatized = json.load(f)['lemmas']
else:
    model = spacy.load('ru_core_news_sm', disable=['ner', 'parser'])
    data_lemmatized = []
    for doc in model.pipe(texts, disable=["tagger", "parser"]):
        data_lemmatized.append([token.lemma_ for token in doc])

    with open("lemmas.json", "w") as fid:
        json.dump({"lemmas": data_lemmatized}, fid)

print(data_lemmatized[0])

['прийти', 'в', 'гость', 'к', 'vadim', 'nareyko', ',', 'партнёр', 'encata', '-', 'engineering', 'catalyst', ',', 'развивающему', 'направление', 'encata', 'soft', '(', 'разработка', 'софт', 'для', 'стартап', ')', '.', 'вадим', '-', 'основатель', 'школа', 'управленческий', 'мастерство', ',', 'ментор', 'год', '(', '2015', ')', ',', 'стартапер', ',', 'сертифицировать', 'scrum', 'master', '.', 'говорить', 'об', 'управление', 'проект', 'в', 'it', ',', 'о', 'программирование', 'и', 'зарплата', 'разработчик', 'в', '90е', ',', 'бережливый', 'производство', ',', 'подготовка', 'и', 'обучение', 'менеджер', ',', 'развитие', 'управленец', ',', 'стартап', ',', 'о', 'том', ',', 'как', 'попасть', 'в', 'акселератор', 'startup', 'wise', 'guys', 'и', 'многом', 'друг', '.', 'и', ',', 'конечно', ',', 'топ-5', 'совет', 'стартап', 'за', '2', 'минута', '.', 'прогулялись', 'по', 'производство', 'encata', ',', 'где', 'создавать', 'mvp', 'для', 'стартап', ',', 'спуститься', 'в', 'бункер', 'хакспейса', 'makeit', '

Очистим тексты от ненужных символов

In [9]:
word_pattern = re.compile("^[а-я]*$")

def remove_symbols(doc):
    return [token for token in doc if word_pattern.match(token)]

data_words = list(map(remove_symbols, data_lemmatized))
print(data_words[0])

['прийти', 'в', 'гость', 'к', 'развивающему', 'направление', 'разработка', 'софт', 'для', 'стартап', 'вадим', 'основатель', 'школа', 'управленческий', 'мастерство', 'ментор', 'год', 'стартапер', 'сертифицировать', 'говорить', 'об', 'управление', 'проект', 'в', 'о', 'программирование', 'и', 'зарплата', 'разработчик', 'в', 'бережливый', 'производство', 'подготовка', 'и', 'обучение', 'менеджер', 'развитие', 'управленец', 'стартап', 'о', 'том', 'как', 'попасть', 'в', 'акселератор', 'и', 'многом', 'друг', 'и', 'конечно', 'совет', 'стартап', 'за', 'минута', 'прогулялись', 'по', 'производство', 'где', 'создавать', 'для', 'стартап', 'спуститься', 'в', 'бункер', 'хакспейса', 'где', 'любой', 'человек', 'без', 'подготовка', 'через', 'день', 'обучение', 'мочь', 'заниматься', 'лазерный', 'резка', 'за', 'в', 'бережливоепроизводство', 'управлениепроектами']


Загрузим русские стоп-слова

In [10]:
stop_words = stopwords.words('russian')
stop_words += ['это', 'свой', 'очень', 'мочь', 'ваш', 'наш']

Определим функцию для удаления стоп-слов

In [11]:
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

Применим функцию для удаления стоп-слов

In [12]:
len(data_words)

74

In [13]:
data_words_nostops = remove_stopwords(data_words)

In [14]:
len(data_words_nostops)

74

In [15]:
id2word = corpora.Dictionary(data_words_nostops)

texts = data_words_nostops

corpus = [id2word.doc2bow(text) for text in texts]

In [16]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('акселератор', 1),
  ('бережливоепроизводство', 1),
  ('бережливый', 1),
  ('бункер', 1),
  ('вадим', 1),
  ('говорить', 1),
  ('год', 1),
  ('гость', 1),
  ('день', 1),
  ('друг', 1),
  ('заниматься', 1),
  ('зарплата', 1),
  ('лазерный', 1),
  ('любой', 1),
  ('мастерство', 1),
  ('менеджер', 1),
  ('ментор', 1),
  ('минута', 1),
  ('многом', 1),
  ('направление', 1),
  ('обучение', 2),
  ('основатель', 1),
  ('подготовка', 2),
  ('попасть', 1),
  ('прийти', 1),
  ('программирование', 1),
  ('прогулялись', 1),
  ('проект', 1),
  ('производство', 2),
  ('развивающему', 1),
  ('развитие', 1),
  ('разработка', 1),
  ('разработчик', 1),
  ('резка', 1),
  ('сертифицировать', 1),
  ('совет', 1),
  ('создавать', 1),
  ('софт', 1),
  ('спуститься', 1),
  ('стартап', 4),
  ('стартапер', 1),
  ('управленец', 1),
  ('управление', 1),
  ('управлениепроектами', 1),
  ('управленческий', 1),
  ('хакспейса', 1),
  ('человек', 1),
  ('школа', 1)]]

# Протестируем моделирование с 20 темами

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=20, 
    random_state=765,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
    )

In [18]:
lda_model.print_topics()

[(0,
  '0.017*"человек" + 0.017*"встреча" + 0.013*"хотеть" + 0.011*"курс" + 0.010*"работать" + 0.010*"дата" + 0.009*"резюме" + 0.008*"год" + 0.008*"сделать" + 0.008*"стать"'),
 (1,
  '0.015*"весь" + 0.013*"обучение" + 0.013*"место" + 0.009*"агентство" + 0.008*"уровень" + 0.007*"управление" + 0.007*"год" + 0.007*"менторство" + 0.007*"попасть" + 0.007*"хотеть"'),
 (2,
  '0.018*"разработка" + 0.011*"сложный" + 0.011*"идея" + 0.007*"работа" + 0.007*"работать" + 0.007*"начать" + 0.007*"собственный" + 0.007*"фриланс" + 0.007*"сделать" + 0.007*"коллега"'),
 (3,
  '0.027*"работа" + 0.023*"программа" + 0.015*"собственный" + 0.015*"зона" + 0.015*"комфорт" + 0.012*"время" + 0.012*"профессиональный" + 0.008*"команда" + 0.008*"окружение" + 0.008*"сообщество"'),
 (4,
  '0.016*"год" + 0.012*"интересный" + 0.010*"работа" + 0.010*"пост" + 0.009*"возможность" + 0.008*"твой" + 0.008*"айти" + 0.008*"армия" + 0.007*"тебе" + 0.006*"резюме"'),
 (5,
  '0.023*"анкета" + 0.013*"стажировка" + 0.010*"каждый" + 0.

Вычислим сложность и согласованность модели

In [19]:
print('Perplexity: ', lda_model.log_perplexity(corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Perplexity:  -7.861011144280269
Coherence Score:  0.3321243746346544


Отобразим результаты моделирования

In [20]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18     0.211047  0.092819       1        1  8.473839
13     0.065645  0.039417       2        1  8.416893
1      0.010138 -0.167770       3        1  7.674991
15    -0.013698  0.085757       4        1  7.142273
0     -0.082915  0.110500       5        1  7.117306
7     -0.123980  0.053909       6        1  6.988952
4     -0.014228 -0.044150       7        1  6.732302
17     0.003101  0.023606       8        1  5.543074
12    -0.013725  0.042972       9        1  5.193489
5     -0.038702 -0.036152      10        1  4.781581
16    -0.020566  0.058889      11        1  4.428626
11    -0.015366 -0.050191      12        1  4.412840
14     0.063194 -0.018273      13        1  4.395911
10     0.054650 -0.091421      14        1  4.062551
9     -0.027901 -0.016717      15        1  3.929423
2      0.036574  0.006152      16        1  2.912044
6     -0.054319 -0.000319      17        1  2.896592
3     -0.023891 -0.018271      18        1  2.514586
19    -0.002747 -0.040953      19        1  1.376125
8     -0.012309 -0.029806      20        1  1.006601, topic_info=            Term       Freq      Total Category  logprob  loglift
89        задача  17.000000  17.000000  Default  30.0000  30.0000
707    программа  16.000000  16.000000  Default  29.0000  29.0000
671        время  23.000000  23.000000  Default  28.0000  28.0000
245         опыт  28.000000  28.000000  Default  27.0000  27.0000
10    заниматься  14.000000  14.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
138       писать   0.348210   9.081297  Topic20  -5.1213   1.3374
712       решить   0.348105   8.418942  Topic20  -5.1216   1.4129
1100    линкедин   0.348080   3.959881  Topic20  -5.1217   2.1671
116        место   0.348077  10.781989  Topic20  -5.1217   1.1654
273           лс   0.348025   3.835027  Topic20  -5.1218   2.1989

[1214 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
728      13  0.735036  автоматизация
1784     18  0.811733          автор
1999      3  0.744005      агентство
1999      8  0.186001      агентство
1848      1  0.872724   адаптируются
...     ...       ...            ...
232      15  0.201069           эфир
232      16  0.201069           эфир
727       2  0.355761       являться
727       6  0.355761       являться
727      18  0.355761       являться

[2057 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 14, 2, 16, 1, 8, 5, 18, 13, 6, 17, 12, 15, 11, 10, 3, 7, 4, 20, 9])

# Подбор количества тем

Подберем количество тем основываясь на согласованности модели

In [21]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(
            corpus=corpus,
            id2word=id2word,
            num_topics=num_topics, 
            random_state=765,
            update_every=1,
            chunksize=100,
            passes=10,
            alpha='auto',
            per_word_topics=True
            )
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [22]:
#model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_nostops, start=1, limit=10, step=1)

In [23]:
# limit=10
# start=1
# step=1
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

Наилучший результат показала модель с 3 темами

In [24]:
def model_scorer(data):
    id2word = corpora.Dictionary(data)

    texts = data

    corpus = [id2word.doc2bow(text) for text in texts]

    lda_model = gensim.models.ldamodel.LdaModel(
        corpus=corpus,
        id2word=id2word,
        num_topics=3, 
        random_state=765,
        update_every=1,
        chunksize=100,
        passes=10,
        alpha='auto',
        per_word_topics=True
        )

    print('Perplexity: ', lda_model.log_perplexity(corpus))

    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)

    return lda_model, corpus, id2word

best_model, best_corpus, best_id2word = model_scorer(data_words_nostops)

Perplexity:  -7.6096167036462665
Coherence Score:  0.2968403866630028


# Использование биграмм и триграмм

Проверим, повлияет ли использование биграмм и триграмм на согласованность модели

In [25]:
bigram = gensim.models.Phrases(data_words_nostops, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words_nostops], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [26]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_bigrams = make_bigrams(data_words_nostops)
data_words_trigrams = make_trigrams(data_words_nostops)

In [27]:
model_scorer(data_words_bigrams)
print('Complete')

Perplexity:  -7.593407662714701
Coherence Score:  0.2970447272697376
Complete


In [28]:
model_scorer(data_words_trigrams)
print('Complete')

Perplexity:  -7.593407662714701


Использование биграмм и триграмм только ухудшили согласованнось

# Анализ наилучшей модели

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(best_model, best_corpus, best_id2word)
vis

In [ ]:
topic_probs = best_model.get_document_topics(best_corpus)
max_prob_topics = list(map(lambda doc_probs: max(doc_probs, key=lambda x: x[1])[0], topic_probs))

In [ ]:
df = pd.DataFrame([list(data_dd["text"]), max_prob_topics]).transpose()
df.columns = ['text', 'topic_label']
df.head(3)

In [ ]:
df.loc[df['topic_label'] == 1].head()

In [ ]:
best_model.show_topic(2)

# Вывод

С имеющимися данными наилучшую согласованность имеет модель с разделением на 3 темы. После разделения на 3 темы было выявлено, что как отдельная тема выделены тексты на украинском языке. Для получения нормальных результатов нужно провести повторное моделирование исключив из данных украинские тексты.

In [ ]:
df.head(3)

In [ ]:
data_dd.head(3)

In [ ]:
data_words_nostops

In [ ]:
display(df.shape)
display(data_dd.shape)
display(len(data_words_nostops))

In [ ]:
id2word = corpora.Dictionary(data_words_nostops)

texts = data_words_nostops

corpus = [id2word.doc2bow(text) for text in texts]